In [13]:
from IPython.display import clear_output
import serial
import csv
import time
import datetime
from jupyterplot import ProgressPlot

In [14]:
arduino_port = "/dev/ttyUSB0"
arduino_baudrate = 115200
arduino_serial = serial.Serial(arduino_port, arduino_baudrate)
file_path = ""


def show_data_with_csv():
    data_header = ["LL", "LR", "DT", "AX", "AZ"]
    pp = ProgressPlot(
        plot_names=["Light Sensor", "LiDAR Distance"], line_names=["L", "R"]
    )
    try:
        with open(
            file_path + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".csv",
            mode="w",
            newline="",
        ) as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(["Time"] + data_header)
            start_time = time.time()
            while time.time() - start_time < 120:
                data = arduino_serial.readline().decode("utf-8").strip()
                if data.count(",") == len(data_header) - 1:
                    try:
                        LL, LR, DT, AX, AZ = map(
                            lambda x: float(x[3:]), data.split(",")
                        )
                        timestamp = datetime.datetime.now().strftime(
                            "%Y-%m-%d %H:%M:%S"
                        )
                        csv_writer.writerow([timestamp, LL, LR, DT, AX, AZ])
                        pp.update([[LL, LR], [DT, DT]])
                    except ValueError as e:
                        print(f"Invalid data format: {data}; {e}")
                else:
                    print(f"Invalid data count: {data}")
                time.sleep(0.1)
            pp.finalize()
    except KeyboardInterrupt:
        arduino_serial.close()


def show_data():
    pp = ProgressPlot(
        plot_names=["Light Sensor", "LiDAR Distance"], line_names=["L", "R"]
    )
    try:
        while True:
            data = arduino_serial.readline().decode("utf-8").strip()
            try:
                LL, LR, DT, AX, AZ = map(lambda x: float(x[3:]), data.split(","))
                pp.update([[LL, LR], [DT, 0]])
            except ValueError as e:
                print(f"Invalid data format: {data}; {e}")
            time.sleep(0.01)
    except Exception:
        arduino_serial.close()
        del pp
        

def show_rps_data():
    pp = ProgressPlot(plot_names=["PWM"], line_names=["A", "B", "C", "D"], height=800, width=1000)
    try:
        while True:
            try:
                data = arduino_serial.readline().decode("utf-8").strip()
            except UnicodeDecodeError:
                continue
            try:
                A, B, C, D = map(float, data.split(","))
                pp.update([[A, B, C, D]])
            except ValueError as e:
                print(f"Invalid data format: {data}; {e}")
            time.sleep(0.01)
    except (Exception,KeyboardInterrupt):
        arduino_serial.close()
        del pp
    finally:
        clear_output()

show_rps_data()